## Разметка платежей по соглашниям с регфондами по видам работы



## Цель и описание проекта

**Задача:**  
Автоматизировать процесс определения **вида финансируемых работ** по тексту **назначения платежа** в соглашениях с региональными фондами. Это позволит сократить время ручной разметки и повысить точность учёта расходов по видам работ.

**Что было сделано:**
- Из большого набора (~66 000 строк за 2020–2025 годы) выделено 2000 строк для ручной разметки.
- Обучена модель логистической регрессии, достигнута высокая точность (F1 ≈ 99.87).
- Проведено несколько итераций активного обучения: модель предсказывает, человек проверяет, далее модель переобучается.
- Выделены случаи, в которых модель не может уверенно сделать прогноз (например, неполные или абстрактные формулировки). Эти данные исключаются из обучения и обрабатываются отдельно.
- Получена итоговая таблица с предсказаниями, готовая для аналитики в разрезе соглашений и видов работ.

**Итог:**  
Создан инструмент для классификации платежей по видам работ с высокой точностью. Он помогает формировать отчёты без участия человека, масштабируется на новые данные, повышает прозрачность процессов и снижает риски ошибок при ручной обработке.

Выводы и рекомендации приведены в конце проекта





### Загрузка и предобработка данных

In [25]:
!python -m spacy download ru_core_news_sm -q
!pip install xlrd -q
!pip install openpyxl -q
!pip install catboost -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 119.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import spacy
import os
import re
import nltk
import joblib
from catboost import CatBoostClassifier

from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder

RANDOM_STATE = 42
TEST_SIZE = 0.2

In [27]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '{:,.2f}'.format(x).replace(',', ' ').replace('.', ','))

In [28]:
df = pd.read_excel(
    r'platezhi_07042025_.xls',
    sheet_name='TDSheet',
    header=0,
    parse_dates =['Регистратор.Дата']
)

df.head()
data_copy = df.copy()

<ipython-input-28-d437433773b2>:1: UserWarning: Parsing dates in %d.%m.%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_excel(


In [ ]:
df.info()

In [ ]:
# Переименовываем столбцы и отбираем только нужные
renaming_dict = {
    'Регистратор.Дата': 'date',
    'Договор контрагента.Номер договора': 'contract_number',
    'Регистратор.Исходный документ.Назначение платежа': 'payment_purpose',
    ' В валюте упр. учета': 'expense',
    'ВИД работы Сводный бюджет': 'work_type',
    'check': 'check' # чек проверки платежа чтобы повторно не проверять
}
df = df[renaming_dict.keys()]
df.rename(columns=renaming_dict, inplace=True)
df.head()

In [ ]:
df[df.duplicated()].head() # Дубликаты убирать не буду

In [ ]:
df['work_type'] = df['work_type'].str.lower()
df['work_type'].value_counts()

### Анализ данных

In [ ]:
# Общие расходы по видам работ
df_pivot = df.groupby(['work_type', 'contract_number'])['expense'].sum().reset_index()
pivot = df_pivot.pivot_table(
    values='expense',
    index='work_type',
    aggfunc='sum',
)
pivot['share_%'] = pivot['expense'] / pivot['expense'].sum() * 100
print(pivot.sort_values(by='expense'))
print('Общие прочие расходы', pivot.query('work_type != "СМР"')['expense'].sum())

In [ ]:
df_pivot['is_other'] = df_pivot['work_type'] != 'СМР'
pivot = df_pivot.pivot_table(
    values='expense',
    index='contract_number',
    aggfunc='sum'
)

other_expense = df_pivot[df_pivot['is_other']].groupby('contract_number')['expense'].sum().rename('other_expense')

result = pivot.join(other_expense).fillna(0)
result['other_share'] = result['other_expense'] / result['expense'] * 100

plt.figure(figsize=(10,4))
plt.hist(result.query('other_share < 50')['other_share'], bins=100, edgecolor='black')
plt.title('Распределение доли прочих расходов по соглашениям')
plt.xlabel('Доля прочих расходов')
plt.ylabel('Количество соглашений')
plt.grid(True)
plt.tight_layout()
plt.show()

### Обучение модели без BERT

#### Подготовка данных

In [35]:
# выделение тестовой выборки
data = df.query('date > "2021-01-01" and expense < 0')
X = data['payment_purpose']
y = data['work_type']
X = X.str.lower()

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=y
)

In [36]:
nlp = spacy.load("ru_core_news_sm", disable=['parser', 'ner'])
num_cores = os.cpu_count()


# функция лемматизации
def lemmatize(texts):
    lemmatized_texts = []
    for doc in nlp.pipe(texts, batch_size=100, n_process=num_cores):
        lemmatized_texts.append(" ".join([token.lemma_ for token in doc]))
    return lemmatized_texts


# функция очистки текстов
def clear_text(text):
    text = re.sub(r'[^а-яА-ЯёЁ ]', ' ', text)
    clean_text = " ".join(text.split())
    return clean_text

# загрузка стоп-слов
nltk.download('stopwords')
stopwords = list(nltk_stopwords.words('russian'))
stopwords.extend(['ндс', 'фзп', 'кф', 'рб', 'фб', 'сумма', 'p', 'р', 'мо', 'соглфзп' ])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
%%time
is_download = True
# предобработка или загрузка данных
if is_download:
    X_train = pd.read_pickle('X_train_lemmas.pkl')
    X_test = pd.read_pickle('X_test_lemmas.pkl')
    X = pd.read_pickle('X_lemmas.pkl')
else:
    # очистка, лемматизация и сохранение тренировочной выборки
    X_train = X_train.apply(clear_text) # очистка текста
    X_train = X_train.to_frame()
    X_train['lemm_text'] = lemmatize(X_train['payment_purpose']) # лемматизация текста
    X_train.to_pickle('X_train_lemmas.pkl')

    # очистка лемматизация и сохранение тестовой выборки
    X_test = X_test.apply(clear_text) # очистка текста
    X_test = X_test.to_frame()
    X_test['lemm_text'] = lemmatize(X_test['payment_purpose']) # лемматизация текста
    X_test.to_pickle('X_test_lemmas.pkl')

    X = X.apply(clear_text) # очистка текста
    X = X.to_frame()
    X['lemm_text'] = lemmatize(X['payment_purpose']) # лемматизация
    X.to_pickle('X_lemmas.pkl')



CPU times: user 126 ms, sys: 16.7 ms, total: 143 ms
Wall time: 142 ms


In [38]:
# кодирование меток
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

#### Пайплан

In [39]:
# создание пайплайна

pipe_final = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2), stop_words=stopwords)),
    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
])

param_grid = [
    # словарь для модели LogisticRegression()
    {
        'models': [LogisticRegression(
                      random_state=RANDOM_STATE,
                      max_iter=100,
                      class_weight='balanced',
                      penalty='l2'
                  )],
        'models__C': [0.01, 0.1, 1, 10, 100],
        'models__solver': ['liblinear', 'saga', 'lbfgs'],
        'models__class_weight': ['balanced'],
        'models__penalty': ['l2']
    },

    # # Добавление elasticnet регуляризации

    {
        'models': [LogisticRegression(
                      random_state=RANDOM_STATE,
                      max_iter=100,
                      class_weight='balanced',
                      solver='saga'
                  )],
        'models__C': [0.01, 0.1, 1, 10, 100],
        'models__penalty': ['elasticnet'],  # ElasticNet регуляризация
        'models__l1_ratio': [0.1, 0.5, 0.7, 0.9, 1.0],  # Соотношение l1 и l2
        'models__class_weight': ['balanced'],  # Балансировка классов
    },

    {
        'models': [CatBoostClassifier(verbose=0, random_state=RANDOM_STATE)],
        'models__iterations': [100, 200],
        'models__learning_rate': [0.03, 0.1],
        'models__depth': [4, 6, 8],
    }

    # словарь для модели RandomForestClassifier()
    {
        'models': [RandomForestClassifier(
                      random_state=RANDOM_STATE,
                      class_weight='balanced',
                  )],
        'models__n_estimators': range(50, 100),  # Количество деревьев в лесу
        'models__max_depth': range(2, 10),      # Максимальная глубина дерева
    },

    # словарь для модели GradientBoostingClassifier
    {
        'models': [GradientBoostingClassifier(
                      random_state=RANDOM_STATE
                  )],
        'models__n_estimators': range(50, 100, 10),
        'models__learning_rate': [0.01, 0.1, 0.2],
        'models__max_depth': range(2, 5),
    },

    # словарь для модели XGBoost
    {
        'models': [XGBClassifier(
                      tree_method='gpu_hist',   # основной параметр
                      predictor='gpu_predictor',
                      gpu_id=0,
                      random_state=RANDOM_STATE,
                      use_label_encoder=False,
                      eval_metric='logloss'
                  )],
        'models__n_estimators': range(50, 100, 10),
        'models__learning_rate': [0.01, 0.1, 0.2],
        'models__max_depth': range(2, 5),
        'models__scale_pos_weight': [1, 10, 25],  # Для дисбаланса классов
    },

]

In [40]:
%%time
# поиск оптимальных параметров c помощью RandomizedSearchCV
stratified_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

randomized_search = RandomizedSearchCV(
    pipe_final,
    param_grid,
    cv=stratified_cv,
    scoring='f1_macro',
    random_state=RANDOM_STATE,
    n_jobs=-1,
    n_iter=10,
    verbose=2,
)

if is_download:
    best_model = joblib.load('best_logistic_model.pkl')
else:
    randomized_search.fit(X_train['lemm_text'], y_train)
    best_model = randomized_search.best_estimator_
    joblib.dump(best_model, 'best_logistic_model.pkl')
    print('Лучшая модель и её параметры:\n\n', randomized_search.best_estimator_)
    print ('Метрика лучшей модели на тренировочной выборке:', randomized_search.best_score_)

CPU times: user 43.4 ms, sys: 0 ns, total: 43.4 ms
Wall time: 43 ms


In [41]:
# Метрика лучшей модели на тестовой выборке

y_pred = best_model.predict(X_test['lemm_text'])
f1 = f1_score(y_test, y_pred, average='macro')
print ('Метрика лучшей модели на тестовой выборке:', f1);

Метрика лучшей модели на тестовой выборке: 0.9986325066165143


Модель показала хорошую метрику на тестовой выборке.

### Подготовка данных для ручной проверки меток

In [ ]:
y_pred = best_model.predict(X['lemm_text'])
y_pred = pd.DataFrame(y_pred)
y_pred = pd.Series(le.inverse_transform(y_pred),index=X.index, name='prediction')
y_pred.head()

In [ ]:
X['predict'] = y_pred
df['predict'] = X['predict']
df['predict'] = df['predict'].fillna(df['work_type'])
df.sample(5)

In [44]:
# сохранение непроверенных платежей для ручной проверки
df.query('work_type != predict and check == 1').to_excel('mismatched_predictions.xlsx', index=False)

In [45]:
# расчет уверенности модели в разметке
proba = best_model.predict_proba(X['lemm_text'])
classes = best_model.classes_

df_proba = pd.DataFrame(proba, columns=classes, index=X.index)
df_proba['confidence'] = df_proba[classes].max(axis=1)
df_proba.query('confidence <=0.95').shape

(116, 9)

In [46]:
# формирование реестра для ручной проверки платежей в которых модель не уверена
df['confidence'] = df_proba['confidence']
df.query('confidence <= 0.97 and work_type == predict and check != 1').to_excel('mismatched_predictions_2.xlsx', index=False)

In [47]:
# реестр случайных платежей для финальной проверки
df.query('check != 1').sample(500).to_excel('sample_500_prediction.xlsx', index=False)

### Прогнозирование на обученной модели

In [48]:
df_new = pd.read_excel(
    r'p.xlsx',
    sheet_name='TDSheet',
    header=0,
    parse_dates =['Регистратор.Дата']
)

<ipython-input-48-fb4a8972c9e2>:1: UserWarning: Parsing dates in %d.%m.%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_new = pd.read_excel(


In [49]:
data_copy = data_copy.rename({'ВИД работы Сводный бюджет' : 'Вид платежа'}, axis=1)

In [ ]:
# Выбираем список признаков для объединения:
common_columns = list(set(df_new.columns) & set(data_copy.columns))


# Теперь объединяем, предсказываем только по новым признакам
df_result = df_new.merge(
    data_copy[['Регистратор.Исходный документ', 'Вид платежа']],
    on='Регистратор.Исходный документ',
    how='left'
)
df_result.sample(1)

In [51]:
data = df_result[df_result['Вид платежа'].isnull()]
data = data['Регистратор.Исходный документ.Назначение платежа'].apply(clear_text) # очистка текста
data = data.to_frame()
data['lemm_text'] = lemmatize(data['Регистратор.Исходный документ.Назначение платежа']) # лемматизация текста
data.to_pickle('data_lemmas.pkl')

In [52]:
y_pred = best_model.predict(data['lemm_text'])
y_pred = pd.DataFrame(y_pred)
y_pred = pd.Series(le.inverse_transform(y_pred),index=data.index, name='prediction')
df_result['Вид платежа'] = df_result['Вид платежа'].fillna(y_pred)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [53]:
proba = best_model.predict_proba(data['lemm_text'])
classes = best_model.classes_

data_proba = pd.DataFrame(proba, columns=classes, index=data.index)
data_proba['confidence'] = df_proba[classes].max(axis=1)
df_result['confidence'] = data_proba['confidence']

In [54]:
df_result.to_excel('data_25.04.2025.xlsx')

In [56]:
df_result['Вид платежа'] = df_result['Вид платежа'].str.lower()
pivot = df_result.groupby(['Договор контрагента.Номер договора', 'Вид платежа'])[' В валюте упр. учета'].sum()

pivot.to_excel('data_pivot.xlsx')

## Выводы

Разработка модели для автоматической разметки назначений платежей по видам работ показала высокую эффективность и практическую применимость. Использованная стратегия — комбинирование ручной разметки, активного обучения и отбора «неуверенных» предсказаний — позволила достичь **F1-метрики 99.87%** на тестовой выборке и сохранить высокое качество на полном датасете.

Модель:
- Успешно масштабируется на новые данные;
- Устраняет необходимость ручной классификации большинства строк;
- Сохраняет интерпретируемость (логистическая регрессия позволяет отслеживать, какие слова влияют на решение).

Проект реализован с учётом:
- Исключения шума (платежей, которые не поддаются автоматической классификации);
- Интеграции в бизнес-процесс: итоговый датасет позволяет строить агрегированную аналитику по соглашениям и видам работ;
- Возможности контроля качества: включён цикл обратной связи — модель указывает на сомнительные примеры для повторной проверки.

### Что можно улучшить:
- **Дополнительно использовать методы NLP**: например, TF-IDF или BERT-эмбеддинги для повышения устойчивости к вариативности текстов;
- **Добавить категориальные признаки** (например, регион, источник финансирования, номер соглашения), что может повысить точность;
- **Развить пайплайн до полноценного сервиса**, с возможностью периодического обновления модели и логгирования качества;
- **Интегрировать в BI-систему**, где пользователи смогут видеть распределение платежей по видам работ в реальном времени, включая неуверенные случаи.

### Общий вывод:
Проект успешно решает задачу классификации по слабоструктурированному тексту. Это позволяет использовать модель в производственной среде для оптимизации рутинной аналитики и повышения точности бюджетного планирования. Подход может быть масштабирован и на другие направления с текстовыми источниками.
